In [1]:
"""An example involving Energy Flow Networks (EFNs), which were introduced 
in [1810.05165](https://arxiv.org/abs/1810.05165). The [`EFN`](../docs/
archs/#efn) class is used to construct the network architecture.
The output of the example is a plot of the ROC curves obtained by
the EFN as well as the jet mass and constituent multiplicity 
observables.
"""

# standard library imports
from __future__ import absolute_import, division, print_function

# standard numerical library imports
import numpy as np

# energyflow imports
import energyflow as ef
from energyflow.archs import EFN
from energyflow.datasets import qg_jets
from energyflow.utils import data_split, to_categorical

# attempt to import sklearn
try:
    from sklearn.metrics import roc_auc_score, roc_curve
except:
    print('please install scikit-learn in order to make ROC curves')
    roc_curve = False

# attempt to import matplotlib
try:
    import matplotlib.pyplot as plt
except:
    print('please install matploltib in order to make plots')
    plt = False
    
import tensorflow as tf

In [5]:
def foo():
    """lol catz"""
    pass

bar = foo
bar.__doc__ = 'hey'

In [6]:
foo.__doc__

'hey'

In [2]:
train, val, test = 75000, 10000, 15000

# load data
X, y = qg_jets.load(train + val + test, pad=False, ncol=3)

# convert labels to categorical
Y = to_categorical(y, num_classes=2)

print('Loaded quark and gluon jets')

# preprocess by centering jets and normalizing pts
for x in X:
    yphi_avg = np.average(x[:,1:3], weights=x[:,0], axis=0)
    x[:,1:3] -= yphi_avg
    x[:,0] /= x[:,0].sum()

print('Finished preprocessing')

# do train/val/test split 
(z_train, z_val, z_test, 
 p_train, p_val, p_test,
 Y_train, Y_val, Y_test) = data_split(np.asarray([x[:,0] for x in X], dtype='O'),
                                      np.asarray([x[:,1:3] for x in X], dtype='O'),
                                      Y, val=val, test=test)
X_train, X_val, X_test = data_split(X, val=val, test=test) 

print('Done train/val/test split')

Loaded quark and gluon jets
Finished preprocessing
Done train/val/test split


In [2]:
"""An example using Energy Flow Networks (EFNs), which were introduced in
[1810.05165](https://arxiv.org/abs/1810.05165), to classify quark and gluon
jets. The [`EFN`](../docs/archs/#efn) class is used to construct the network
architecture. This example is meant to highlight the usafe of PFNs with
Tensorflow datasets, in particular the function `tf_point_cloud_dataset` which
helpfully formats things in the proper way.  The output of the example is a
plot of the ROC curves obtained by the EFN as well as the jet mass and
constituent multiplicity observables.
"""

# standard library imports
from __future__ import absolute_import, division, print_function

# standard numerical library imports
import numpy as np

# energyflow imports
import energyflow as ef
from energyflow.archs import EFN, tf_point_cloud_dataset
from energyflow.datasets import qg_jets
from energyflow.utils import data_split, to_categorical

from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

################################### SETTINGS ##################################
# the commented values correspond to those in 1810.05165
###############################################################################

# data controls, can go up to 2000000 total for full dataset
train, val, test = 75000, 10000, 15000
# train, val, test = 1000000, 200000, 200000

# network architecture parameters
Phi_sizes, F_sizes = (100, 100, 128), (100, 100, 100)
# Phi_sizes, F_sizes = (100, 100, 256), (100, 100, 100)

# network training parameters
num_epoch = 5
batch_size = 500

###############################################################################

# load data
X, y = qg_jets.load(train + val + test, pad=False, ncol=3)

# convert labels to categorical
Y = to_categorical(y, num_classes=2)

print('Loaded quark and gluon jets')

# preprocess by centering jets and normalizing pts
for x in X:
    yphi_avg = np.average(x[:,1:3], weights=x[:,0], axis=0)
    x[:,1:3] -= yphi_avg
    x[:,0] /= x[:,0].sum()

print('Finished preprocessing')

# do train/val/test split 
(z_train, z_val, z_test, 
 p_train, p_val, p_test,
 Y_train, Y_val, Y_test) = data_split(np.asarray([x[:,0] for x in X], dtype='O'),
                                      np.asarray([x[:,1:] for x in X], dtype='O'),
                                      Y, val=val, test=test)

print('Done train/val/test split')
print('Model summary:')

# build architecture
efn = EFN(input_dim=2, Phi_sizes=Phi_sizes, F_sizes=F_sizes)

# train model
d_train = tf_point_cloud_dataset([(z_train, p_train), Y_train], batch_size)
d_val = tf_point_cloud_dataset([(z_val, p_val), Y_val], batch_size)
print(d_train)

Loaded quark and gluon jets
Finished preprocessing
Done train/val/test split
Model summary:
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
phats_input (InputLayer)        [(None, None, 2)]    0                                            
__________________________________________________________________________________________________
tdist_0 (TimeDistributed)       (None, None, 100)    300         phats_input[0][0]                
__________________________________________________________________________________________________
activation (Activation)         (None, None, 100)    0           tdist_0[0][0]                    
__________________________________________________________________________________________________
tdist_1 (TimeDistributed)       (None, None, 100)    10100       activation[0][0]             

In [3]:
for x in d_train.take(5).as_numpy_iterator():
    print(x[0][0].shape, x[0][1].shape, x[1].shape)

(500, 101) (500, 101, 2) (500, 2)
(500, 102) (500, 102, 2) (500, 2)
(500, 114) (500, 114, 2) (500, 2)
(500, 118) (500, 118, 2) (500, 2)
(500, 113) (500, 113, 2) (500, 2)


In [3]:
ef.archs.tf_point_cloud_dataset([[X_train]])

<ZipDataset shapes: (((None, 3),),), types: ((tf.float32,),)>

In [8]:
d = ef.archs.tf_point_cloud_dataset([[z_train, p_train], Y_train])

In [10]:
len(d)

TypeError: dataset length is unknown.

In [23]:
d = tf.data.Dataset.from_generator(ef.archs.tf_gen(y), (tf.float32), ())

In [39]:
dir(d)

['_GeneratorState',
 '__abstractmethods__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_add_variable_with_custom_getter',
 '_apply_options',
 '_as_serialized_graph',
 '_checkpoint_dependencies',
 '_consumers',
 '_deferred_dependencies',
 '_flat_shapes',
 '_flat_structure',
 '_flat_types',
 '_functions',
 '_gather_saveables_for_checkpoint',
 '_graph',
 '_graph_attr',
 '_handle_deferred_dependencies',
 '_has_captured_ref',
 '_input_dataset',
 '_inputs',
 '_list_extra_dependencies_for_serialization',
 '_list_functions_for_serialization',
 '_lookup_dependency',
 '_map_func',
 '_map_reso

In [30]:
isinstance(d, tf.data.Dataset)

True

In [24]:
list(d.take(5).as_numpy_iterator())

[1.0, 1.0, 1.0, 1.0, 1.0]

In [15]:
tf.DType(np.float32)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'type'

In [10]:
d_X_train = tf.data.Dataset.from_generator(ef.archs.tf_gen(z_train, p_train),
                                           (tf.float32, tf.float32),
                                           ((None,), (None, 3)))

In [16]:
tf.as_dtype('float32')

tf.float32

In [5]:
d_z_train = tf.data.Dataset.from_generator(ef.archs.tf_gen(z_train), tf.float32, (None,))
d_p_train = tf.data.Dataset.from_generator(ef.archs.tf_gen(p_train), tf.float32, (None, 2))
d_Y_train = tf.data.Dataset.from_tensor_slices(Y_train.astype('float32'))

In [9]:
d_t = tf.data.Dataset.zip((tf.data.Dataset.zip((d_z_train, d_p_train)), d_Y_train))

In [10]:
d_train.element_spec

((TensorSpec(shape=(None, None), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, None, 2), dtype=tf.float32, name=None)),
 TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))

In [11]:
d_t.element_spec

((TensorSpec(shape=(None,), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)),
 TensorSpec(shape=(2,), dtype=tf.float32, name=None))

In [14]:
d_t.padded_batch(10).element_spec

((TensorSpec(shape=(None, None), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, None, 2), dtype=tf.float32, name=None)),
 TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))

In [15]:
for x in d_t.padded_batch(10).prefetch(2).take(5).as_numpy_iterator():
    print(x[0][0].shape, x[0][1].shape, x[1].shape)

(10, 72) (10, 72, 2) (10, 2)
(10, 75) (10, 75, 2) (10, 2)
(10, 63) (10, 63, 2) (10, 2)
(10, 55) (10, 55, 2) (10, 2)
(10, 62) (10, 62, 2) (10, 2)


In [6]:
# build architecture
efn = ef.archs.efn.EFN(input_dim=2, Phi_sizes=(100, 100, 128), F_sizes=(100, 100, 100))

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
phats_input (InputLayer)        [(None, None, 2)]    0                                            
__________________________________________________________________________________________________
tdist_0 (TimeDistributed)       (None, None, 100)    300         phats_input[0][0]                
__________________________________________________________________________________________________
activation (Activation)         (None, None, 100)    0           tdist_0[0][0]                    
__________________________________________________________________________________________________
tdist_1 (TimeDistributed)       (None, None, 100)    10100       activation[0][0]                 
_______________________________________________________________________________________

In [27]:
d_X_train = tf.data.Dataset.from_generator(ef.archs.tf_gen(X_train), tf.float32, (None, 3))
d_Y_train = tf.data.Dataset.from_tensor_slices(Y_train.astype('float32'))

#d_train = tf.data.Dataset.zip((d_X_train, d_Y_train))
d_train = tf.data.Dataset.from_generator(ef.archs.tf_gen(X_train, y[:train]),
                                         (tf.float32, tf.float32),
                                         ((None, 3), ()))

In [28]:
efn.fit(d_train.padded_batch(100), epochs=2, steps_per_epoch=None)

Epoch 1/2


AssertionError: in user code:

    /home/pkomiske/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /home/pkomiske/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/pkomiske/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/pkomiske/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/pkomiske/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/pkomiske/.local/lib/python3.8/site-packages/tensorflow/pyth

In [17]:
for x in d_z_train.padded_batch(5).take(5).as_numpy_iterator():
    print(x.shape)

(5, 75)
(5, 62)
(5, 79)
(5, 71)
(5, 83)


In [8]:
list(d_p_train.take(2).as_numpy_iterator())

[array([[-0.3004548 ,  0.18000665],
        [ 0.25595946,  0.21914275],
        [ 0.19012931, -0.27834758],
        [ 0.22757659,  0.23368613],
        [ 0.25591214,  0.1861676 ],
        [ 0.27219489,  0.06283828],
        [ 0.21943663,  0.03113066],
        [ 0.17531635,  0.04260233],
        [ 0.0648605 ,  0.15472177],
        [-0.06192421,  0.14590517],
        [-0.15585459, -0.00907615],
        [-0.1395952 , -0.06344516],
        [ 0.11944982, -0.06689146],
        [-0.05746589,  0.0937828 ],
        [ 0.02805894,  0.10240761],
        [-0.05741437, -0.10040385],
        [-0.09678105,  0.04224598],
        [-0.01384912,  0.07198374],
        [-0.0132647 , -0.07073488],
        [ 0.02660296,  0.04084296],
        [-0.0360684 ,  0.0348671 ],
        [ 0.04377095,  0.01773009],
        [ 0.03054393, -0.0431101 ],
        [ 0.01464761, -0.05085827],
        [-0.04123637, -0.02153384],
        [-0.01931104, -0.03783215],
        [ 0.02740436,  0.01964118],
        [-0.02657702,  0.009

In [26]:
len(list(d.take(10).as_numpy_iterator()))

InvalidArgumentError: TypeError: `generator` yielded an element that could not be converted to the expected type. The expected type was float64, but the yielded element was (array([1.55805992e-04, 3.09940222e-03, 2.75630667e-03, 4.18105241e-03,
       4.42553043e-03, 4.73962813e-04, 1.34788840e-03, 3.97825052e-03,
       2.06414032e-03, 2.07097368e-03, 2.80241157e-03, 3.07364337e-03,
       4.16936673e-03, 1.02444531e-03, 8.10773558e-04, 2.94415682e-03,
       7.33718182e-03, 3.41485663e-03, 8.88909015e-03, 7.57798702e-03,
       2.77567553e-03, 2.91540280e-04, 6.23680858e-03, 2.55978176e-03,
       5.24034575e-03, 1.77698967e-02, 4.04757623e-03, 3.88675621e-03,
       5.24140409e-03, 4.44374262e-03, 1.06204548e-03, 3.41156341e-03,
       2.04517878e-02, 5.03235004e-03, 2.91949302e-02, 3.17085406e-02,
       7.72992607e-03, 2.37704276e-02, 1.23474694e-02, 1.33794326e-02,
       1.52389228e-02, 9.09262197e-03, 1.05772026e-01, 2.71705960e-02,
       1.43403527e-01, 9.04701766e-02, 3.0591

In [13]:
for x,y in d.padded_batch(5).take(10).as_numpy_iterator():
    print(x.shape,y.shape)

(5, 57, 4) (5,)
(5, 63, 4) (5,)
(5, 86, 4) (5,)
(5, 67, 4) (5,)
(5, 68, 4) (5,)
(5, 47, 4) (5,)
(5, 62, 4) (5,)
(5, 64, 4) (5,)
(5, 64, 4) (5,)
(5, 59, 4) (5,)


In [15]:
zip(X)

In [2]:
################################### SETTINGS ###################################

# data controls, can go up to 2000000 total for full dataset
train, val, test = 75000, 10000, 15000

# network architecture parameters
ppm_sizes = (100, 100, 128)
dense_sizes = (100, 100, 100)

# network training parameters
num_epoch = 25
batch_size = 500

################################################################################

# load data
X, y = qg_jets.load(train + val + test)

# ignore pid information
X = X[:,:,:3]

# convert labels to categorical
Y = to_categorical(y, num_classes=2)

print('Loaded quark and gluon jets')

# preprocess by centering jets and normalizing pts
for x in X:
    mask = x[:,0] > 0
    yphi_avg = np.average(x[mask,1:3], weights=x[mask,0], axis=0)
    x[mask,1:3] -= yphi_avg
    x[mask,0] /= x[:,0].sum()

print('Finished preprocessing')

# do train/val/test split 
(z_train, z_val, z_test, 
 p_train, p_val, p_test,
 Y_train, Y_val, Y_test) = data_split(X[:,:,0], X[:,:,1:], Y, val=val, test=test)

print('Done train/val/test split')

Loaded quark and gluon jets
Finished preprocessing
Done train/val/test split


In [4]:
# build architecture
pfn = ef.archs.efn.PFNAdditionalFInputs(input_dim=2, Phi_sizes=(100, 100, 128), F_sizes=(100, 100, 100),
                                        additional_F_inputs_dim=10)

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, None, 2)]    0                                            
__________________________________________________________________________________________________
tdist_0 (TimeDistributed)       (None, None, 100)    300         input[0][0]                      
__________________________________________________________________________________________________
activation_7 (Activation)       (None, None, 100)    0           tdist_0[0][0]                    
__________________________________________________________________________________________________
tdist_1 (TimeDistributed)       (None, None, 100)    10100       activation_7[0][0]               
_______________________________________________________________________________________

In [5]:
type(pfn)

energyflow.archs.efn.PFNAdditionalFInputs

In [6]:
a = [0, 1, 2, 3, 4, 5]
a.insert(1, 'p')
a

[0, 'p', 1, 2, 3, 4, 5]

In [4]:
efn.tensors

[<tf.Tensor 'mask/mul:0' shape=(None, None) dtype=float32>,
 <tf.Tensor 'zs_input:0' shape=(None, None) dtype=float32>,
 <tf.Tensor 'phats_input:0' shape=(None, None, 2) dtype=float32>,
 <tf.Tensor 'tdist_0/Reshape_1:0' shape=(None, None, 100) dtype=float32>,
 <tf.Tensor 'activation/Relu:0' shape=(None, None, 100) dtype=float32>,
 <tf.Tensor 'sum/Squeeze:0' shape=(None, 100) dtype=float32>,
 <tf.Tensor 'output/BiasAdd:0' shape=(None, 2) dtype=float32>,
 <tf.Tensor 'activation_1/Softmax:0' shape=(None, 2) dtype=float32>]

In [ ]:

# train model
efn.fit([z_train, p_train], Y_train,
          epochs=num_epoch,
          batch_size=batch_size,
          validation_data=([z_val, p_val], Y_val),
          verbose=2)

# get predictions on test data
preds = efn.predict([z_test, p_test], batch_size=1000)

# get ROC curve if we have sklearn
if roc_curve:
    efn_fp, efn_tp, threshs = roc_curve(Y_test[:,1], preds[:,1])

    # get area under the ROC curve
    auc = roc_auc_score(Y_test[:,1], preds[:,1])
    print()
    print('EFN AUC:', auc)
    print()

    # make ROC curve and filter plot if we have matplotlib
    if plt:

        # some nicer plot settings 
        plt.rcParams['font.family'] = 'serif'
        plt.rcParams['figure.autolayout'] = True

        fig, axes = plt.subplots(1, 2, figsize=(8,4))

        ######################### ROC Curve Plot #########################

        # get multiplicity and mass for comparison
        masses = np.asarray([ef.ms_from_p4s(ef.p4s_from_ptyphims(x).sum(axis=0)) for x in X])
        mults = np.asarray([np.count_nonzero(x[:,0]) for x in X])
        mass_fp, mass_tp, threshs = roc_curve(Y[:,1], -masses)
        mult_fp, mult_tp, threshs = roc_curve(Y[:,1], -mults)

        # plot the ROC curves
        axes[0].plot(efn_tp, 1-efn_fp, '-', color='black', label='EFN')
        axes[0].plot(mass_tp, 1-mass_fp, '-', color='blue', label='Jet Mass')
        axes[0].plot(mult_tp, 1-mult_fp, '-', color='red', label='Multiplicity')

        # axes labels
        axes[0].set_xlabel('Quark Jet Efficiency')
        axes[0].set_ylabel('Gluon Jet Rejection')

        # axes limits
        axes[0].set_xlim(0, 1)
        axes[0].set_ylim(0, 1)

        # make legend and show plot
        axes[0].legend(loc='lower left', frameon=False)

        ######################### Filter Plot #########################

        # plot settings
        R, n = 0.4, 100
        colors = ['Reds', 'Oranges', 'Greens', 'Blues', 'Purples', 'Greys']
        grads = np.linspace(0.45, 0.55, 4)

        # evaluate filters
        X, Y, Z = efn.eval_filters(R, n=n)

        # plot filters
        for i,z in enumerate(Z):
            axes[1].contourf(X, Y, z/np.max(z), grads, cmap=colors[i%len(colors)])
        
        axes[1].set_xticks(np.linspace(-R, R, 5))
        axes[1].set_yticks(np.linspace(-R, R, 5))
        axes[1].set_xticklabels(['-R', '-R/2', '0', 'R/2', 'R'])
        axes[1].set_yticklabels(['-R', '-R/2', '0', 'R/2', 'R'])
        axes[1].set_xlabel('Translated Rapidity y')
        axes[1].set_ylabel('Translated Azimuthal Angle phi')
        axes[1].set_title('Energy Flow Network Latent Space', fontdict={'fontsize': 10})
    
        plt.show()